## Import

In [38]:
import spacy
nlp = spacy.load('/home/alana/Documentos/NLP/UDEMY/en_core_web_sm-2.2.0/en_core_web_sm/en_core_web_sm-2.2.0')

In [39]:
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)

## Criando padrões
Na literatura, a frase 'solar power' pode aparecer como uma ou duas palavras, com ou sem hífen. Nesta seção, desenvolveremos um matcher chamado 'SolarPower' que encontra todos os três:

In [23]:
pattern1 = [{'LOWER': 'solarpower'}]
pattern2 = [{'LOWER': 'solar'}, {'LOWER': 'power'}]
pattern3 = [{'LOWER': 'solar'}, {'IS_PUNCT': True}, {'LOWER': 'power'}]

matcher.add('SolarPower', None, pattern1, pattern2, pattern3)

## Aplicando o matcher a um objeto Doc

    A indústria de energia solar continua a crescer conforme a demanda por energia solar aumenta. Carros movidos a energia solar estão ganhando popularidade.

In [24]:
doc = nlp(u'The Solar Power industry continues to grow as demand for solarpower increases. Solar-power cars are gaining popularity.')

In [25]:
found_matches = matcher(doc)

print(found_matches)

[(8656102463236116519, 1, 3), (8656102463236116519, 10, 11), (8656102463236116519, 13, 16)]


    matcher retorna uma lista de tuplas. Cada tupla contém um ID para a correspondência, com tokens de início e fim que mapeiam para o span doc [Inicio: Fim]

In [26]:
for match_id, start, end in found_matches:
    string_id = nlp.vocab.strings[match_id]
    span = doc[start:end]
    print(match_id, string_id, start, end, span.text)

8656102463236116519 SolarPower 1 3 Solar Power
8656102463236116519 SolarPower 10 11 solarpower
8656102463236116519 SolarPower 13 16 Solar-power


## Configurando opções de padrão e quantificadores
Você pode tornar as regras de token opcionais passando um argumento `'OP': '*'`. Isso nos permite simplificar nossa lista de padrões:

In [27]:
# Redefina os padrões:
pattern1 = [{'LOWER': 'solarpower'}]
pattern2 = [{'LOWER': 'solar'}, {'IS_PUNCT': True, 'OP':'*'}, {'LOWER': 'power'}]


# Remova os padrões antigos para evitar duplicação:
matcher.remove('SolarPower')

# Adicione o novo conjunto de padrões ao matcher 'SolarPower':
matcher.add('SolarPower', None, pattern1, pattern2)

In [28]:
found_matches = matcher(doc)
print(found_matches)

[(8656102463236116519, 1, 3), (8656102463236116519, 10, 11), (8656102463236116519, 13, 16)]


Isso encontrou os dois padrões de duas palavras, com e sem o hífen!

Os seguintes quantificadores podem ser passados ​​para a chave `'OP'`:
<table><tr><th>OP</th><th>Descrição</th> </tr>

<tr> <td> <span> \! </span> </td> <td> Negue o padrão, exigindo que corresponda exatamente 0 vezes </td> </tr>
<tr> <td> <span>? </span> </td> <td> Torne o padrão opcional, permitindo que corresponda 0 ou 1 vez </td> </tr>
<tr> <td> <span> \ + </span> </td> <td> Exigir que o padrão corresponda 1 ou mais vezes </td> </tr>
<tr> <td> <span> \ * </span> </td> <td> Permitir que o padrão corresponda zero ou mais vezes </td> </tr>
</table>

## Cuidado com os lemas!
Se quiséssemos combinar "solar power" e "solar powered", pode ser tentador procurar o * lema * de "powered" e esperar que seja "power". Isso não é sempre o caso! O lema do * adjetivo * 'powered' ainda é 'powered':

In [29]:
pattern1 = [{'LOWER': 'solarpower'}]
pattern2 = [{'LOWER': 'solar'}, {'IS_PUNCT': True, 'OP':'*'}, {'LEMMA': 'power'}]

# Remova os padrões antigos para evitar duplicação:
matcher.remove('SolarPower')

# Adicione o novo conjunto de padrões ao matcher 'SolarPower':
matcher.add('SolarPower', None, pattern1, pattern2)

    Energia solar movimenta carros movidos a energia solar.

In [30]:
doc2 = nlp(u'Solar-powered energy runs solar-powered cars.')

In [31]:
found_matches = matcher(doc2)
print(found_matches)

[(8656102463236116519, 0, 3), (8656102463236116519, 5, 8)]


In [32]:
pattern1 = [{'LOWER': 'solarpower'}]
pattern2 = [{'LOWER': 'solar'}, {'IS_PUNCT': True, 'OP':'*'}, {'LOWER': 'power'}]
pattern3 = [{'LOWER': 'solarpowered'}]
pattern4 = [{'LOWER': 'solar'}, {'IS_PUNCT': True, 'OP':'*'}, {'LOWER': 'powered'}]

matcher.remove('SolarPower')

matcher.add('SolarPower', None, pattern1, pattern2, pattern3, pattern4)

In [33]:
found_matches = matcher(doc2)
print(found_matches)

[(8656102463236116519, 0, 3), (8656102463236116519, 5, 8)]


## Outros atributos de token
Além dos lemas, há uma variedade de atributos de token que podemos usar para determinar as regras de correspondência:
<table><tr><th>Atributo</th><th>Descrição</th> </tr>

<tr> <td> <span> `ORTH` </span> </td> <td> O texto literal exato de um token </td> </tr>
<tr> <td> <span> `LOWER` </span> </td> <td> A forma em minúsculas do texto do token </td> </tr>
<tr> <td> <span> `LENGTH` </span> </td> <td> O comprimento do texto do token </td> </tr>
<tr> <td> <span> `IS_ALPHA`,` IS_ASCII`, `IS_DIGIT` </span> </td> <td> O texto do token consiste em caracteres alfanuméricos, caracteres ASCII, dígitos </td> </tr>
<tr> <td> <span> `IS_LOWER`,` IS_UPPER`, `IS_TITLE` </span> </td> <td> O texto do token está em minúsculas, maiúsculas, titlecase </td> </tr>
<tr> <td> <span> `IS_PUNCT`,` IS_SPACE`, `IS_STOP` </span> </td> <td> Token é pontuação, espaço em branco, palavra de interrupção </td> </tr>
<tr> <td> <span> `LIKE_NUM`,` LIKE_URL`, `LIKE_EMAIL` </span> </td> <td> O texto do token se parece com um número, URL, e-mail </td> </tr>
<tr> <td> <span> `POS`,` TAG`, `DEP`,` LEMMA`, `SHAPE` </span> </td> <td> A tag de classe gramatical simples e estendida do token , rótulo de dependência, lema, forma </td> </tr>
<tr> <td> <span> `ENT_TYPE` </span> </td> <td> O rótulo da entidade do token </td> </tr>

</table>

## Token curinga
Você pode passar um dicionário vazio `{}` como um curinga para representar ** qualquer token **. Por exemplo, você pode querer recuperar hashtags sem saber o que pode seguir o caractere `#`:
> `[{'ORTH': '#'}, {}]`

# PhraseMatcher
Na seção acima, usamos padrões de token para realizar a correspondência baseada em regras. Um método alternativo - e geralmente mais eficiente - é fazer a correspondência em listas de terminologia. Neste caso, usamos PhraseMatcher para criar um objeto Doc a partir de uma lista de frases e passá-lo para o `matcher`.

In [45]:
nlp = spacy.load('/home/alana/Documentos/NLP/UDEMY/en_core_web_sm-2.2.0/en_core_web_sm/en_core_web_sm-2.2.0')

In [46]:
from spacy.matcher import PhraseMatcher
matcher = PhraseMatcher(nlp.vocab)

In [92]:
import unicodedata
import re
import pandas as pd


def read_clean_texto(path):
    
    with open(path, 'r') as f:
        texto = f.read()
        texto = texto.lower()
        texto = re.sub(r'\n+|\t+', ' ' ,texto)
        texto = texto.encode('ascii', 'ignore')
        texto = texto.decode("utf-8")
        texto = unicodedata.normalize('NFD', texto)
        
        return str(texto) 

In [94]:
path = 'reaganomics.txt'
doc3 = nlp(texto)

In [95]:
# Primeiro, crie uma lista de frases correspondentes:
phrase_list = ['voodoo economics', 'supply-side economics', 'trickle-down economics', 'free-market economics']

# Em seguida, converta cada frase em um objeto Doc:
phrase_patterns = [nlp(text) for text in phrase_list]

# Passe cada objeto Doc para o matcher (observe o uso do asterisco!):
matcher.add('VoodooEconomics', None, *phrase_patterns)

# Construa uma lista de correspondências:
matches = matcher(doc3)

In [97]:
# match_id, inicio, fim
matches

[(3473369816841043438, 40, 44),
 (3473369816841043438, 48, 52),
 (3473369816841043438, 53, 55),
 (3473369816841043438, 60, 64),
 (3473369816841043438, 664, 668),
 (3473369816841043438, 2960, 2964)]

    As primeiras quatro correspondências são onde esses termos são usados na definição de Reaganomics:

In [98]:
doc3[:70]

reaganomics https://en.wikipedia.org/wiki/reaganomics reaganomics (a portmanteau of [ronald] reagan and economics attributed to paul harvey)[1] refers to the economic policies promoted by u.s. president ronald reagan during the 1980s. these policies are commonly associated with supply-side economics, referred to as trickle-down economics or voodoo economics by political opponents, and free-market economics by political advocates. the four

# Viewing Partidas
    Existem algumas maneiras de buscar o texto em torno de uma correspondência. O mais simples é pegar uma fatia de tokens do documento que seja mais larga do que a correspondência:

In [101]:
doc3[665:685]  # Observe que a quinta partida começa no doc3 [664]

supply-side economics movement, which formed in opposition to keynesian demand-stimulus economics. this movement produced some

In [102]:
doc3[2975:2995]

brackets, as that extra money for the wealthy could trickle along to low-income groups.[67] federal income

    Outra maneira é primeiro aplicar o sentencizer ao Doc e, em seguida, iterar através das frases até o match point:

In [103]:
# Construa uma lista de frases
sents = [sent for sent in doc3.sents]

# Na próxima seção, veremos que as frases contêm valores de token inicial e final:
print(sents[0].start, sents[0].end)

0 34


In [105]:
# Repita a lista de frases até que o valor final da frase exceda um valor inicial de correspondência:
for sent in sents:
    if matches[4][1] < sent.end:  # esta é a quinta partida, que começa no doc3 [664]
        print(sent)
        break

at the same time he attracted a following from the supply-side economics movement, which formed in opposition to keynesian demand-stimulus economics.


https://spacy.io/usage/linguistic-features#section-rule-based-matching